In [2]:


def file_as_bytes(file):
    with file:
        return file.read()
def file_md5sum(file):
    import hashlib
    try:
        return hashlib.md5(file_as_bytes(open(file, 'rb'))).hexdigest()
    except:
        print ("md5sum error", file)
        return False
    
def push_panda_table_sql(connection, table_name, df):
    
    mycursor=connection.cursor()
    mycursor.execute("SHOW TABLES LIKE '"+table_name+"'")
    result = mycursor.fetchone()
    if not result:
        cols = "`,`".join([str(i) for i in df.columns.tolist()])
        db_to_crete = "CREATE TABLE `"+table_name+"` ("+' '.join(["`"+x+"` REAL," for x in df.columns.tolist()])[:-1]+")"
        print ("[Table {:s} created into SQL Server]".format(table_name))
        mycursor = connection.cursor()
        mycursor.execute(db_to_crete)

    cols = "`,`".join([str(i) for i in df.columns.tolist()])

    for i,row in df.iterrows():
        sql = "INSERT INTO `"+table_name+"` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
        mycursor.execute(sql, tuple(row.astype(str)))
        connection.commit()

    mycursor.close()


    
def update_sql_value(connection, table_name, row_element, row_element_condition, 
                     colum_element, value, verbose=False):
    if isinstance(value, str):
        svalue="\""+value+"\""
    else:
        svalue=str(value)
    if isinstance(row_element_condition, str):
        scondition="\""+row_element_condition+"\""
    else:
        scondition=str(row_element_condition)
    sql = "UPDATE `"+table_name+"` SET `"+colum_element+"` = "+ \
    svalue+" WHERE `"+row_element+"` = "+scondition+";"
    if verbose: print(sql)
    try:
        mycursor = connection.cursor()
        mycursor.execute(sql)
        connection.commit()
        if verbose: print(mycursor.rowcount, "Update done")
        mycursor.close()
        return 0
    except:
        return 1

def daq_sql_cennection(verbose=False):
    import mysql.connector
    import os
    try:
        connection = mysql.connector.connect(
          host=os.environ['MYSQL_IP'],
          user=os.environ['MYSQL_USER'],
          password=os.environ['MYSQL_PASSWORD'],
          database=os.environ['MYSQL_DATABASE'],
          port=int(os.environ['MYSQL_PORT'])
        )
        if verbose: print(connection)
        return connection
    except:
        return False
    
def daq_update_runlog_replica_checksum(connection, run_number, md5sum, verbose=False):
    if verbose: print("md5sum: ", md5sum)
    return update_sql_value(connection, table_name="Runlog", row_element="run_number", 
                     row_element_condition=run_number, 
                     colum_element="file_checksum", value=md5sum, 
                     verbose=verbose)
def daq_update_runlog_replica_tag(connection, run_number, TAG, verbose=False):
    if verbose: print("TAG: ", TAG)
    return update_sql_value(connection, table_name="Runlog", row_element="run_number", 
                     row_element_condition=run_number, 
                     colum_element="file_s3_tag", value=TAG, 
                     verbose=verbose)

def daq_update_runlog_replica_status(connection, run_number, storage, status=-1, verbose=False):
    if storage=="local":
        storage="storage_local_status"
    elif storage=="cloud":
        storage="storage_cloud_status"
    elif storage=="tape":
        storage="storage_tape_status"
    else:
        return 1
    if verbose: print("Storage: "+storage)
    return update_sql_value(connection, table_name="Runlog", row_element="run_number", 
                     row_element_condition=run_number, 
                     colum_element=storage, value=status, 
                     verbose=verbose)
     
    

In [27]:

verbose=True
run_number=268
file_path="/data01/data/run00268.mid.gz"
md5sum=file_md5sum(file_path)
print (md5sum)
connection = daq_sql_cennection(verbose)
daq_update_runlog_replica_status(connection, run_number, storage="local", status=3, verbose=verbose)
daq_update_runlog_replica_checksum(connection, run_number, md5sum, verbose=verbose)
daq_update_runlog_replica_tag(connection, run_number, TAG="LAB", verbose=verbose)

eb5188e7570438d9b47a91565b46b89a
Storage: storage_local_status
UPDATE `Runlog` SET `storage_local_status` = 3 WHERE `run_number` = 268;
0 Update done
md5sum:  eb5188e7570438d9b47a91565b46b89a
UPDATE `Runlog` SET `file_checksum` = "eb5188e7570438d9b47a91565b46b89a" WHERE `run_number` = 268;
0 Update done
TAG:  LAB
UPDATE `Runlog` SET `file_s3_tag` = "LAB" WHERE `run_number` = 268;
0 Update done


0

In [2]:
def get_content_md5(data):
    import base64
    import hashlib
    digest = hashlib.md5(data).digest()
    return base64.b64encode(digest)

In [3]:
!ls /data01/data/

220321.hst  221107.idf		 run00182.mid.gz      run00513.mid.gz
220321.idf  221107.idx		 run00183.mid.crc32c  run00514.mid.crc32c
220321.idx  221108.hst		 run00183.mid.gz      run00514.mid.gz
220322.hst  221108.idf		 run00184.mid.crc32c  run00515.mid.crc32c
220322.idf  221108.idx		 run00184.mid.gz      run00515.mid.gz
220322.idx  221109.hst		 run00185.mid.crc32c  run00516.mid.crc32c
220323.hst  221109.idf		 run00185.mid.gz      run00516.mid.gz
220323.idf  221109.idx		 run00186.mid.crc32c  run00517.mid.crc32c
220323.idx  221110.hst		 run00186.mid.gz      run00517.mid.gz
220330.hst  221110.idf		 run00187.mid.crc32c  run00518.mid.crc32c
220330.idf  221110.idx		 run00187.mid.gz      run00518.mid.gz
220330.idx  221111.hst		 run00188.mid.crc32c  run00519.mid.crc32c
220331.hst  221111.idf		 run00188.mid.gz      run00519.mid.gz
220331.idf  221111.idx		 run00189.mid.crc32c  run00520.mid.crc32c
220331.idx  221112.hst		 run00189.mid.gz      run00520.mid.gz
220401.hst  221112.idf		 run00190.mid.crc3

In [4]:
get_content_md5("/data01/data/run00759.mid.gz")

TypeError: Unicode-objects must be encoded before hashing

In [6]:
import boto3
boto3.utils.compute_md5("/data01/data/run00759.mid.gz")

AttributeError: module 'boto3.utils' has no attribute 'compute_md5'

In [2]:
def file_as_bytes(file):
    with file:
        return file.read()
file_path="/data01/data/run00759.mid.gz"
def md5sum(file_path):
    import hashlib
    return hashlib.md5(file_as_bytes(open(full_path, 'rb'))).hexdigest()
md5sum(file_path)

'ed0732f7f826b3b6fd99076629489b69'

In [12]:
! md5sum /data01/data/run00759.mid.gz

ed0732f7f826b3b6fd99076629489b69  /data01/data/run00759.mid.gz


In [6]:
def daq_sql_cennection(verbose=False):
    import mysql.connector
    import os
    try:
        connection = mysql.connector.connect(
          host=os.environ['MYSQL_IP'],
          user=os.environ['MYSQL_USER'],
          password=os.environ['MYSQL_PASSWORD'],
          database=os.environ['MYSQL_DATABASE'],
          port=int(os.environ['MYSQL_PORT'])
        )
        if verbose: print(connection)
        return connection
    except:
        return False

def daq_not_on_tape_runs(connection, verbose=False):
    import numpy as np
    sql = "SELECT * FROM `Runlog` WHERE DATEDIFF(CURRENT_TIMESTAMP, start_time) < 30 \
    AND `storage_tape_status` < 1 AND `storage_cloud_status` = 1;"
    mycursor = connection.cursor()
    mycursor.execute(sql)
    value = mycursor.fetchall()
    if verbose: print(mycursor.rowcount)
    mycursor.close()
    try:
        runs = np.array(list(zip(*value))[0])
    except:
        runs = []
    return runs
def daq_not_on_local_runs(connection, verbose=False):
    import numpy as np
    sql = "SELECT * FROM `Runlog` WHERE DATEDIFF(CURRENT_TIMESTAMP, start_time) < 30  AND `storage_local_status` = 0;"
    mycursor = connection.cursor()
    mycursor.execute(sql)
    value = mycursor.fetchall()
    if verbose: print(mycursor.rowcount)
    mycursor.close()
    try:
        runs = np.array(list(zip(*value))[0])
    except:
        runs = []
    return runs

In [5]:
verbose = True
connection = daq_sql_cennection(verbose)
daq_not_on_tape_runs(connection, verbose=verbose)

18


array([5282, 5283, 5284, 5285, 5286, 5287, 5288, 5289, 5290, 5291, 5292,
       5293, 5294, 5295, 5296, 5297, 5298, 5299])

In [1]:
import cygno as cy
verbose = True
connection = cy.daq_sql_cennection(verbose)
runN=5703
cy.daq_read_runlog_replica_status(connection, runN, storage="local", verbose=verbose)
daq_not_on_local_runs(connection, verbose=False)

Storage: storage_local_status
SELECT `storage_local_status` FROM `Runlog` WHERE `run_number` = 5703;
1 Update done


NameError: name 'daq_not_on_local_runs' is not defined

In [ ]:
import os 
import datetime
import cygno as cy
connection = cy.daq_sql_cennection(False)
in_path =  '/data01/data/'
os.chdir(in_path)
tail_of_file = 200
file_in_dir = sorted(filter(os.path.isfile, os.listdir(in_path)), key=os.path.getmtime)
print(len(file_in_dir))
if len(file_in_dir)<tail_of_file:
    tail_of_file=len(file_in_dir)
for i, file_in in enumerate(file_in_dir):
    if file_in.endswith('.mid'):
        print (file_in)
for i, file_in in enumerate(file_in_dir):
    if file_in.endswith('.mid'):
        print (file_in)

In [2]:
def files_in_dir(in_path, ftype='mid.gz', tail_of_file=200, verbose=False):
    os.chdir(in_path)
    file_in_dir = sorted(filter(os.path.isfile, os.listdir(in_path)), key=os.path.getmtime)

    if len(file_in_dir)<tail_of_file:
        tail_of_file=len(file_in_dir)
    file_in_dir = file_in_dir[-tail_of_file:]
    file_list = []
    for i, file_in in enumerate(file_in_dir):
        if file_in.endswith(ftype):
            file_list.append(file_in)
    return file_list

for i, file_in in enumerate(files_in_dir(in_path, ftype='mid.gz', tail_of_file=200, verbose=False)):
    if file_in.endswith('.mid'):
        runN = int(file_in.split('run')[-1].split('.mid')[0])
        if cy.daq_read_runlog_replica_status(connection, runN, storage="local", verbose=False)==0:
            dtime = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            print('{:s} Compressing file: {:s}'.format(dtime, file_in))

NameError: name 'in_path' is not defined

In [3]:
file_in = files_in_dir(in_path, ftype='mid', tail_of_file=200, verbose=False)

NameError: name 'in_path' is not defined

In [5]:
md5sum = os.system('md5sum ' + in_path + file_in[0])

849123f25d3d123beaea448464bb0778  /data01/data/run05791.mid


In [6]:
def md5sum_file(filein):
    import subprocess
    command = '/bin/md5sum '+ filein
    status, output = subprocess.getstatusoutput(command)
    if status:
        out = False
    else:
        out = output.split(" ")[0]
    return out

In [7]:
md5sum_file(in_path + file_in[0])

'849123f25d3d123beaea448464bb0778'

In [8]:
cy.run_info_logbook(run=5791, sql=True, verbose=False)

,run_number,run_description,start_time,exposure_sec,GEM3_V,GEM2_V,GEM1_V,T2_V,T1_V,DRIFT_V,...,PMT4_V,HV_STATE,stop_time,number_of_events,storage_local_status,storage_cloud_status,storage_tape_status,file_size,file_checksum,file_s3_tag
5251,5791,"S003:DATA:BKG, with pmt Trg: LVL2, 2mV thr, ve...",2022-12-01 17:19:14,0.3,420,420,420,500,500,960,...,770,1,NaT,NaN,-1,-1,-1,-1,None,None


In [9]:
nevent = cy.run_info_logbook(run=5790, sql=True, verbose=False).number_of_events.values[0]

In [10]:
nevent

401.0

In [11]:
if nevent>0:
    print("pippo:)

SyntaxError: EOL while scanning string literal (1903443.py, line 2)

In [ ]:
def daq_run_info(connection, run, verbose=False):
    import numpy as np
    import pandas as pd
    import mysql.connector
    sql = "SELECT * FROM `Runlog` WHERE `run_number` ="+str(run)+";"
    return pd.read_sql(sql, connection)

In [ ]:
daq_run_info(connection, 5063, verbose=False)

In [ ]:
def daq_run_info(connection, run, verbose=False):
    import pandas as pd
    import mysql.connector
    sql = "SELECT * FROM `Runlog` WHERE `run_number` ="+str(run)+";"
    mycursor = connection.cursor()
    mycursor.execute(sql)
    value = mycursor.fetchall()
    if verbose: print(mycursor.rowcount)
    mycursor.close()
    try:
        runs = np.array(value)[0]
    except:
        runs = []
    return runs

daq_run_info(connection, 5036, verbose=False)

In [8]:
value  = cy.daq_run_info(connection, 5810, verbose=False).number_of_events.values[0]
if value == None:
    value = 0
print (value)

0


/home/standard/.local/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [7]:

print(value)
value = (if value  0)

None


In [11]:
"run05812.mid.gz".split(".")[0]+".mid.crc32c"

'run05812.mid.crc32c'

In [13]:
def allert_mail(subject, body, mail_to, verbose):
    import os
    tmail = "echo \""+body+"\" | mutt -s \""+subject+"\" "+mail_to
    if verbose: print (tmail)
    os.system(tmail)

In [14]:
allert_mail("subject", "body", "giovanni.mazzitelli@lnf.infn.it", True)

echo "body" | mutt -s "subject" giovanni.mazzitelli@lnf.infn.it


In [46]:
def files_in_dir(in_path, ftype='mid.gz', tail_of_file=200, verbose=False):
    import os
    file_in_dir = sorted(filter(lambda file_in: file_in.endswith(ftype), os.listdir(in_path)),
                     key=os.path.getmtime)
    file_in_dir = file_in_dir[-tail_of_file:]
    return file_in_dir

In [53]:
files_in_dir('/data01/data/', ftype='mid.gz', tail_of_file=10000, verbose=False)

['run00138.mid.gz',
 'run00139.mid.gz',
 'run00140.mid.gz',
 'run00141.mid.gz',
 'run00142.mid.gz',
 'run00143.mid.gz',
 'run00144.mid.gz',
 'run00145.mid.gz',
 'run00146.mid.gz',
 'run00147.mid.gz',
 'run00148.mid.gz',
 'run00149.mid.gz',
 'run00150.mid.gz',
 'run00151.mid.gz',
 'run00152.mid.gz',
 'run00153.mid.gz',
 'run00154.mid.gz',
 'run00155.mid.gz',
 'run00156.mid.gz',
 'run00157.mid.gz',
 'run00158.mid.gz',
 'run00159.mid.gz',
 'run00240.mid.gz',
 'run00241.mid.gz',
 'run00242.mid.gz',
 'run00243.mid.gz',
 'run00244.mid.gz',
 'run00245.mid.gz',
 'run00246.mid.gz',
 'run00247.mid.gz',
 'run00248.mid.gz',
 'run00249.mid.gz',
 'run00250.mid.gz',
 'run00251.mid.gz',
 'run00252.mid.gz',
 'run00253.mid.gz',
 'run00254.mid.gz',
 'run00255.mid.gz',
 'run00256.mid.gz',
 'run00257.mid.gz',
 'run00258.mid.gz',
 'run00259.mid.gz',
 'run00260.mid.gz',
 'run00261.mid.gz',
 'run00262.mid.gz',
 'run00263.mid.gz',
 'run00264.mid.gz',
 'run00265.mid.gz',
 'run00266.mid.gz',
 'run00267.mid.gz',


In [44]:
import os
ftype=".mid.gz"
os.chdir('/data01/data/')
file_in_dir = sorted(filter(lambda file_in: file_in.endswith(ftype), os.listdir('/data01/data/')),
                     key=os.path.getmtime)

In [45]:
file_in_dir

['run00138.mid.gz',
 'run00139.mid.gz',
 'run00140.mid.gz',
 'run00141.mid.gz',
 'run00142.mid.gz',
 'run00143.mid.gz',
 'run00144.mid.gz',
 'run00145.mid.gz',
 'run00146.mid.gz',
 'run00147.mid.gz',
 'run00148.mid.gz',
 'run00149.mid.gz',
 'run00150.mid.gz',
 'run00151.mid.gz',
 'run00152.mid.gz',
 'run00153.mid.gz',
 'run00154.mid.gz',
 'run00155.mid.gz',
 'run00156.mid.gz',
 'run00157.mid.gz',
 'run00158.mid.gz',
 'run00159.mid.gz',
 'run00240.mid.gz',
 'run00241.mid.gz',
 'run00242.mid.gz',
 'run00243.mid.gz',
 'run00244.mid.gz',
 'run00245.mid.gz',
 'run00246.mid.gz',
 'run00247.mid.gz',
 'run00248.mid.gz',
 'run00249.mid.gz',
 'run00250.mid.gz',
 'run00251.mid.gz',
 'run00252.mid.gz',
 'run00253.mid.gz',
 'run00254.mid.gz',
 'run00255.mid.gz',
 'run00256.mid.gz',
 'run00257.mid.gz',
 'run00258.mid.gz',
 'run00259.mid.gz',
 'run00260.mid.gz',
 'run00261.mid.gz',
 'run00262.mid.gz',
 'run00263.mid.gz',
 'run00264.mid.gz',
 'run00265.mid.gz',
 'run00266.mid.gz',
 'run00267.mid.gz',


In [2]:
def files_in_dir(in_path, ftype='mid.gz', tail_of_file=100, verbose=False):
    import os
    os.chdir(in_path)
    file_in_dir = sorted(filter(lambda file_in: file_in.endswith(ftype), os.listdir(in_path)),
                     key=os.path.getmtime)
    file_in_dir = file_in_dir[-tail_of_file:]
    return file_in_dir
ftype=".mid"
files_in_dir(in_path='/data01/data/', ftype=ftype, tail_of_file=100, verbose=False)

['run05831.mid',
 'run05837.mid',
 'run05973.mid',
 'run06050.mid',
 'run06088.mid',
 'run06089.mid',
 'run06090.mid']

In [4]:
import cygno as cy
verbose = True
connection = cy.daq_sql_cennection(verbose)
cy.daq_run_info(connection, 6090, verbose=verbose).number_of_events.values

/home/standard/.local/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


array([402.])